In [1]:
!pip install flask
!pip install nltk
!pip install urllib3
!pip install bs4
!pip install plotly
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\saish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Importing all the libraries

In [1]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

# for extracting data from finviz
finviz_url = 'https://finviz.com/quote.ashx?t='

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saish\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Getting HTML Headlines from FinViz Using Beautiful Soup

In [50]:
def get_news(ticker):
    url = finviz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    return news_table

ticker = 'TSLA'
amz = get_news(ticker)
amz # pure html code, data is not parsed yet

<table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer" id="news-table" width="100%">
<tr class="cursor-pointer" onclick="trackAndOpenNews(event, 'Investor\u0027s Business Daily', 'https://finance.yahoo.com/m/93a3843c-94a6-3551-9b17-6a956f6ac989/tesla-stock-charges-up-after.html');">
<td align="right" width="130">
            Jun-20-23 04:13PM
        </td>
<td align="left">
<div class="news-link-container">
<div class="news-link-left">
<a class="tab-link-news" href="https://finance.yahoo.com/m/93a3843c-94a6-3551-9b17-6a956f6ac989/tesla-stock-charges-up-after.html" rel="nofollow" target="_blank">Tesla Stock Charges Up After Rivian Signs Off On Supercharger Deal</a>
</div>
<div class="news-link-right">
<span>(Investor's Business Daily)</span> <span class="body-table-news-gain">+5.34%</span></div></div></td></tr>
<tr class="cursor-pointer" onclick="trackAndOpenNews(event, 'Investor\u0027s Business Daily', 'https://finance.yahoo.com/m/f765d21c-713e-36ac-807f-0adbb346

## Parsing News into DF

In [51]:
def parse_news(news_table):
    parsed_news = []
    
    for x in news_table.findAll('tr'):
        # check if 'a' tag is present in the 'tr' tag
        if x.a is not None:
            # read the text from 'a' tag into text
            text = x.a.get_text() 
            # splite text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element
            if len(date_scrape) == 1:
                time = date_scrape[0]
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            # Append ticker, date, time and headline as a list to the 'parsed_news' list
            parsed_news.append([date, time, text])        
    # Set column names
    columns = ['date', 'time', 'headline']
    # Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
    parsed_news_df = pd.DataFrame(parsed_news, columns=columns)        
    # Create a pandas datetime object from the strings in 'date' and 'time' column
    parsed_news_df['datetime'] = pd.to_datetime(parsed_news_df['date'] + ' ' + parsed_news_df['time'])
    
    return parsed_news_df


In [52]:
parsed_news_df = parse_news(amz)


C:\Users\saish\AppData\Local\Temp\ipykernel_16388\1631053543.py:25: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [53]:
parsed_news_df

,date,time,headline,datetime
0,Jun-20-23,04:13PM,Tesla Stock Charges Up After Rivian Signs Off ...,2023-06-20 16:13:00
1,Jun-20-23,04:07PM,Tesla Stock Hit 8-Month High Friday; Cathie Wo...,2023-06-20 16:07:00
2,Jun-20-23,04:06PM,"Rivian to Use Teslas Charging Network, Follows...",2023-06-20 16:06:00
3,Jun-20-23,04:00PM,"US STOCKS-S&P 500 ends lower, pausing rally wi...",2023-06-20 16:00:00
4,Jun-20-23,03:58PM,Tesla Recruits Another Rival to Use Its Superc...,2023-06-20 15:58:00
...,...,...,...,...
95,Jun-16-23,11:42AM,UPDATE 2-Musk says vehicle autonomy is 'main d...,2023-06-16 11:42:00
96,Jun-16-23,11:15AM,Elon Musk repeats call for artificial intellig...,2023-06-16 11:15:00
97,Jun-16-23,11:05AM,Musk: Tesla's value is based primarily on auto...,2023-06-16 11:05:00
98,Jun-16-23,10:54AM,Elon Musk: seeing all-time highs in terms of T...,2023-06-16 10:54:00


## Scores for Sentiment Analysis

In [54]:
def score_news(parsed_news_df):
    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    
    # Iterate through the headlines and get the polarity scores using vader
    scores = parsed_news_df['headline'].apply(vader.polarity_scores).tolist()

    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)

    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = parsed_news_df.join(scores_df, rsuffix='_right')        
    parsed_and_scored_news = parsed_and_scored_news.set_index('datetime')    
    parsed_and_scored_news = parsed_and_scored_news.drop(['date', 'time'], axis = 1)          
    parsed_and_scored_news = parsed_and_scored_news.rename(columns={"compound": "sentiment_score"})
    #parsed_and_scored_news.set_index('datetime', inplace=True)
    return parsed_and_scored_news

parsed_and_scored_news = score_news(parsed_news_df)
parsed_and_scored_news.head()

,headline,neg,neu,pos,sentiment_score
datetime,,,,,
2023-06-20 16:13:00,Tesla Stock Charges Up After Rivian Signs Off ...,0.174,0.826,0.0,-0.2732
2023-06-20 16:07:00,Tesla Stock Hit 8-Month High Friday; Cathie Wo...,0.000,1.000,0.0,0.0000
2023-06-20 16:06:00,"Rivian to Use Teslas Charging Network, Follows...",0.000,1.000,0.0,0.0000
2023-06-20 16:00:00,"US STOCKS-S&P 500 ends lower, pausing rally wi...",0.167,0.833,0.0,-0.2960
2023-06-20 15:58:00,Tesla Recruits Another Rival to Use Its Superc...,0.000,1.000,0.0,0.0000


In [55]:
mean_scores = parsed_and_scored_news.drop('headline', axis=1).resample('H').mean()

mean_scores

,neg,neu,pos,sentiment_score
datetime,,,,
2023-06-16 10:00:00,0.121000,0.879000,0.000000,-0.246950
2023-06-16 11:00:00,0.000000,0.767600,0.232400,0.334920
2023-06-16 12:00:00,0.000000,0.756000,0.244000,0.669700
2023-06-16 13:00:00,0.182000,0.716500,0.101500,-0.177700
2023-06-16 14:00:00,0.089500,0.831000,0.079500,-0.038600
...,...,...,...,...
2023-06-20 12:00:00,0.000000,0.851000,0.149000,0.236150
2023-06-20 13:00:00,0.025429,0.910571,0.064000,0.084771
2023-06-20 14:00:00,0.000000,1.000000,0.000000,0.000000


## Hourly Sentiment 

In [60]:
import plotly.graph_objects as go
import numpy as np

def plot_hourly_sentiment(parsed_and_scored_news, ticker):
    # Group by date and ticker columns from parsed_and_scored_news and calculate the mean
    mean_scores = parsed_and_scored_news.drop('headline', axis=1).resample('H').mean()
    
    # Create a new column called 'color' that maps positive sentiment scores to green and negative scores to red
    mean_scores['color'] = np.where(mean_scores['sentiment_score'] > 0, 'green', 'red')

    # Plot a bar chart with plotly, setting the color of each bar based on the 'color' column
    fig = go.Figure(data=go.Bar(x=mean_scores.index, y=mean_scores['sentiment_score'],
                               marker_color=mean_scores['color']))

    # Update the layout to apply a dark theme
    fig.update_layout(
        title=ticker + ' Hourly Sentiment Scores',
        xaxis_title='Date and Time',
        yaxis_title='Sentiment Score',
        plot_bgcolor='#01111b',
        paper_bgcolor='#01111b',
        font_color='#f4f4f4'
    )

    # Add custom labels for positive and negative sentiment scores
    fig.update_layout(
        coloraxis_colorbar=dict(
            title='Sentiment',
            tickvals=[-1, 0, 1],
            ticktext=['Negative', 'Neutral', 'Positive'],
            tickmode='array',
            ticks='outside',
            len=0.6,
            thickness=15,
            tickfont=dict(color='#f4f4f4')
        )
    )

    return fig

fig = plot_hourly_sentiment(parsed_and_scored_news, ticker)
fig.show()


## Daily Sentiment

In [61]:
import plotly.graph_objects as go

def plot_daily_sentiment(parsed_and_scored_news, ticker):
    mean_scores = parsed_and_scored_news.drop('headline', axis=1).resample('D').mean()

    # Plot a bar chart with plotly
    fig = go.Figure(data=go.Bar(x=mean_scores.index, y=mean_scores['sentiment_score'],
                               marker_color='coral'))

    # Update the layout to apply a dark theme
    fig.update_layout(
        title=ticker + ' Daily Sentiment Scores',
        xaxis_title='Date',
        yaxis_title='Sentiment Score',
        plot_bgcolor='#01111b',
        paper_bgcolor='#01111b',
        font_color='#f4f4f4'
    )

    return fig

fig = plot_daily_sentiment(parsed_and_scored_news, ticker)
fig.show()
